# Calculation of Sea Ice Area and Extent

In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import datetime
import os
import glob
import re
import dask
print(datetime.datetime.now())

2023-08-10 12:01:02.564888


In [95]:
#N.B. MIROC6 requires more memory locally
from dask_jobqueue import PBSCluster
from dask.distributed import Client

cluster = PBSCluster(cores    = 1,
                     memory   = '5GB',
                     queue    = 'casper',
                     walltime = '00:05:00',
                     project  = 'UCUB0084',
                    )

cluster.scale(10)
client = Client(cluster)
client

/glade/u/ssg/ch/usr/jupyterhub/envs/npl-3.7.9/dav/lib/python3.7/site-packages/distributed/node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 35647 instead
  http_address["port"], self.http_server.port


Client Scheduler: tcp://10.12.206.49:40154 Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/cwpowell/proxy/35647/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [2]:
xr_new_time = xr.open_dataarray(
    '/glade/work/cwpowell/low-frequency-variability/raw_data/'\
    +'datetime64_1850_2100_monthly.nc')

lat_names = {
    'ACCESS-ESM1-5':'latitude', 'CESM2-LENS':'lat', 'CanESM5':'latitude',
    'EC-Earth3':'latitude', 'MIROC6':'latitude', 'MPI-ESM1-2-LR':'latitude',     
    'CanESM2':'lat', 'CESM1':'lat', 'GFDL-ESM2M':'lat', 'MPI-ESM1':'lat',
}
 
x_y_names = {
    'ACCESS-ESM1-5':['j','i'], 'CESM2-LENS':['nj','ni'], 'CanESM5':['j','i'], 
    'EC-Earth3':['j','i'], 'MIROC6':['y','x'], 'MPI-ESM1-2-LR':['j','i'], 
    'CanESM2':['lat','lon'], 'CESM1':['j','i'], 'GFDL-ESM2M':['lat','lon'],
    'MPI-ESM1':['j','i'],
}

model_center = {
    'EC-Earth3':'EC-Earth-Consortium', 'CanESM5':'CCCma', 'CESM2-LENS':'NCAR',
    'MIROC6':'MIROC', 'ACCESS-ESM1-5':'CSIRO', 'MPI-ESM1-2-LR':'MPI-M',
    'CanESM2':'canesm2_lens', 'CESM1':'cesm_lens', 
    'GFDL-ESM2M':'gfdl_esm2m_lens', 'MPI-ESM1':'mpi_lens'
}

In [55]:
#reduce areacello file to 30N
#used cdo gridarea to make CanESM2 and GFDL-ESM2M from sic files
orig_areacello = xr.open_dataset(
    '/glade/work/cwpowell/Data/CMIP5_areacello/CanESM2_gridarea.nc')

areacello_30N = orig_areacello.where(orig_areacello['lat']>30)
areacello_30N = areacello_30N.rename({'cell_area':'areacello'})

areacello_30N.to_netcdf('/glade/work/cwpowell/low-frequency-variability/'\
                        'raw_data/masie_masks/areacello_CanESM2_30N.nc')
#for CanESM2 and GFDL-ESM2M make the masiemask, CESM1 and MPI-ESM1 are the 
#same as for the CMIP6 GCM, so just copy over that mask already made
#in low-frequency-variability project

In [3]:
def regional_calc_dask(model_name, mem, areacello_with_nan, region_mask,
                       hist_paths, fut_paths):
    '''
    Calculate regional and pan-Arctic SIA, and SIE as well as regional SIC, 
    from glade files of concentration (siconc or siconca) from historical 
    and future RCP8.5 for CMIP5 or SSP370 for CMIP6 simulations.
    Regions based on NSIDC MASIE regions. 

    Parameters
    ----------
    model_name: string
        The name of the model e.g. CanESM5
    mem: string
        Variant ID e.g. 'r1ip1f1' where the letters correspond to:
        r<>: realisation (i.e. ensemble member),
        i<>: initialisation method, p<>: physics, f<>: forcing
    areacello_with_nan: xarray dataarray
        areacello or areacella with latitudes below 30N set to np.nan
    region_mask: xarray dataarray
        dataarray with the same coordinates and dimensions as the sea ice 
        variable data but with values corresponding to region or np.nan for 
        outside of the regions domain
    hist_paths: list of strings
        A list of the absolute paths for all historical simulations for a given
        GCM and variant ID
    fut_paths: list of strings
        A list of the absolute paths for all future simulations for a given
        GCM and variant ID

    Returns
    ----------
        xarray.Dataset with variables of:
        regional SIA, regional SIE, regional average SIC, pan-Arctic SIA
        pan-Arctic SIE
    '''  
    
    ########################### load the data files ############################
    #load the SIC data and select above 30N        
    if model_name in ['CanESM2','CESM1','GFDL-ESM2M']:
        SIC = xr.open_mfdataset(hist_paths, combine='nested', 
            concat_dim='time', chunks={'time':120}).sortby('time')
    else:
        hist_data = xr.open_mfdataset(hist_paths, combine='nested', 
                                      concat_dim='time', chunks={'time':120})

        fut_data =  xr.open_mfdataset(fut_paths, combine='nested', 
                                         concat_dim='time', chunks={'time':120})

        SIC = xr.concat((hist_data, fut_data), dim='time').sortby('time')
    
    #remove concentration values below 30N and convert to fraction from %
    if model_name == 'CESM2-LENS':
        SIC = SIC['aice'].where(areacello_with_nan, drop=False)
        
    elif model_name in ['CanESM2', 'CESM1', 'MPI-ESM1']:
        SIC = SIC['sic'].where(areacello_with_nan, drop=False)/100
        
    elif model_name == 'GFDL-ESM2M':
        SIC = SIC['sic'].where(areacello_with_nan, drop=False)
        
    else:
        SIC = SIC['siconc'].where(areacello_with_nan, drop=False)/100
  
    #replace time with datetime64 for consistency across models
    if model_name == 'EC-Earth3' and mem not in [
        'r9i1p1f1', 'r6i1p1f1', 'r4i1p1f1', 'r1i1p1f1', 'r11i1p1f1', 
        'r15i1p1f1', 'r13i1p1f1',
    ]:
        #EC-Earth files on /glade often only go back to 1970
        SIC['time'] = xr_new_time.sel(time=slice('1970-01','2100-12'))
        
    elif model_name in ['CanESM2', 'GFDL-ESM2M']:
        SIC['time'] = xr_new_time.sel(time=slice('1950-01','2100-12'))
        
    elif model_name == 'CESM1':
        if mem == 'r1i1p1':
            SIC['time'] = xr_new_time.sel(time=slice('1850-01','2100-12'))
            SIC = SIC.sel(time=slice('1920-01','2100-12'))
        else:
            SIC['time'] = xr_new_time.sel(time=slice('1920-01','2100-12'))
        
    elif model_name == 'MPI-ESM1':
        SIC['time'] = xr_new_time.sel(time=slice('1850-01','2099-12'))
        
    else:
        SIC['time'] = xr_new_time
    
    ################ compute SIA, SIV and average SIC and SIT ##################    
    #only do the calculation where there is sea ice
    SIC = SIC.where(SIC>0)
    
    pan_Arctic_SIA = (SIC * areacello_with_nan).sum(
        x_y_names[model_name][0]).sum(x_y_names[model_name][1])/1e12
    
    SIC_1_0 = SIC.where(SIC>0.15,0)
    SIC_1_0 = SIC_1_0.where(SIC_1_0<0.1,1)
    pan_Arctic_SIE = (SIC_1_0 * areacello_with_nan).sum(
        x_y_names[model_name][0]).sum(x_y_names[model_name][1])/1e12
        
    #calculate regional data
    SIA_regions = []
    SIE_regions = []
    SIC_regions_av = []

    for region_ in np.arange(1,17):
        
        area_region = areacello_with_nan.where(
                region_mask['regions']==region_).sum()
        
        SIC_region = SIC.where((region_mask['regions']==region_))
        SIA_region = (SIC_region * areacello_with_nan).sum(
            x_y_names[model_name][0]).sum(x_y_names[model_name][1])
        
        SIC_1_0_region = SIC_1_0.where((region_mask['regions']==region_))
        SIE_region = (SIC_1_0_region * areacello_with_nan).sum(
            x_y_names[model_name][0]).sum(x_y_names[model_name][1])
        
        SIA_regions.append(SIA_region)
        SIE_regions.append(SIE_region)
        SIC_regions_av.append(SIA_region / area_region)           
    
    ######################### concatenate all regions ##########################
    SIA_regions =  xr.concat((SIA_regions),dim='region')
    SIA_regions['region'] = np.arange(1,17)
    SIE_regions =  xr.concat((SIE_regions),dim='region')
    SIE_regions['region'] = np.arange(1,17)
    SIC_regions_av = xr.concat((SIC_regions_av),dim='region')
    SIC_regions_av['region'] = np.arange(1,17)
    
    final_dataset = xr.Dataset(
            {'regional_SIA':SIA_regions, 'regional_SIC':SIC_regions_av,
             'regional_SIE':SIE_regions, 'Arctic_SIA':pan_Arctic_SIA,
             'Arctic_SIE':pan_Arctic_SIE,
            }
        )
    
    if model_name not in ['CESM2-LENS','CESM1','CanESM2','GFDL-ESM2M','MPI-ESM1']:
        final_dataset = final_dataset.drop('type')
    
    return(final_dataset)

In [5]:
for GCM_ in np.sort(list(model_center.keys())):
    print(datetime.datetime.now(), GCM_)
    
    #load the region masks
    region_mask = xr.open_dataset(
        '/glade/work/cwpowell/low-frequency-variability/raw_data/'\
        +'masie_masks/masiemask_{}.nc'.format(GCM_)
    )

    #load the areacello file
    areacello_ = xr.open_dataset(
        '/glade/work/cwpowell/low-frequency-variability/'\
        +'raw_data/masie_masks/areacello_{}_30N.nc'.format(GCM_)
    )

    if GCM_ == 'CESM2-LENS':
        region_mask = region_mask.rename({'nlat':'nj', 'nlon':'ni'})
        region_mask = region_mask.drop('lat_2').drop('lon_2')
        areacello_ = areacello_.rename({'nlat':'nj', 'nlon':'ni'})
    
    elif GCM_ == 'CESM1':
        region_mask = region_mask.rename({'nlat':'j', 'nlon':'i'})
        region_mask = region_mask.drop('lat_2').drop('lon_2')
        areacello_ = areacello_.rename({'nlat':'j', 'nlon':'i'})
        
    areacello_with_nan = areacello_['areacello']
    
    if GCM_ in ['CanESM2', 'CESM1', 'GFDL-ESM2M', 'MPI-ESM1']:
        var_name = 'sic'
    else:
        var_name = 'siconc'
        
    #obtain a list of the variant IDs for the historical/SSP370 simulations
    if GCM_ == 'CESM2-LENS':
        var_IDs_hist_temp = glob.glob(
            '/glade/campaign/cgd/cesm/CESM2-LE/timeseries/ice/proc/'\
            'tseries/month_1/aice/b.e21.BHISTcmip6.f09_g17.LE2*201412.nc')
        # var_IDs_hist_temp = glob.glob(
        #     '/glade/campaign/cgd/cesm/CESM2-LE/timeseries/ice/proc/'\
        #     'tseries/month_1/aice/b.e21.BHISTsmbb.f09_g17.LE2*201412.nc')
        
        var_IDs_common = []
        for i in var_IDs_hist_temp:
            var_IDs_common.append(i[104:112])
            # var_IDs_common.append(i[103:111])
        
    elif GCM_ in ['CanESM2', 'CESM1', 'GFDL-ESM2M', 'MPI-ESM1']:
        paths_temp = glob.glob('/glade/collections/cdg/data/CLIVAR_LE/'\
                               f'{model_center[GCM_]}/OImon/sic/*rcp85*')
            
        var_IDs_common = np.sort([re.findall('r\d+i\d+p\d',paths_temp[i])[0] 
                                  for i in range(len(paths_temp))])
            
    else:
        var_IDs_hist = os.listdir('/glade/collections/cmip/CMIP6/CMIP/'
                                  f'{model_center[GCM_]}/{GCM_}/historical/')

        var_IDs_fut = os.listdir('/glade/collections/cmip/CMIP6/ScenarioMIP/'
                                    f'{model_center[GCM_]}/{GCM_}/ssp370/')

        var_IDs_common = np.intersect1d(var_IDs_hist, var_IDs_fut)
        
    if GCM_ == 'EC-Earth3': #siconc monthly is unavailable for r5i1p1f1
        var_IDs_common = np.delete(var_IDs_common, 
                                   np.argwhere(var_IDs_common=='r5i1p1f1'))
        #r9i1p1f1 is missing 2078 from /glade
        var_IDs_common = np.delete(var_IDs_common, 
                                   np.argwhere(var_IDs_common=='r9i1p1f1'))
    
    ######## loop through each of the variant IDs ########
    for var_ID in np.sort(var_IDs_common): 
        print(datetime.datetime.now(), var_ID)
        
        if len(glob.glob('/glade/work/cwpowell/ice-free/sea_ice_data/'\
           f'SIA_SIE_{GCM_}_{var_ID}_historical_2100.nc')) > 0:
            continue
       
        ########## get the paths for all the siconc or siconca files ###########
        if GCM_ == 'CESM2-LENS':
            hist_paths = glob.glob(
                '/glade/campaign/cgd/cesm/CESM2-LE/timeseries/ice/proc/tseries/'
                f'month_1/aice/*BHISTcmip6*{var_ID}*.nc')
            
            fut_paths = glob.glob(
                '/glade/campaign/cgd/cesm/CESM2-LE/timeseries/ice/proc/tseries/'
                f'month_1/aice/*370cmip6*{var_ID}*.nc')

#             hist_paths = glob.glob(
#                 '/glade/campaign/cgd/cesm/CESM2-LE/timeseries/ice/proc/tseries/'
#                 f'month_1/aice/*BHISTsmbb*{var_ID}*.nc')
            
#             fut_paths = glob.glob(
#                 '/glade/campaign/cgd/cesm/CESM2-LE/timeseries/ice/proc/tseries/'
#                 f'month_1/aice/*370smbb*{var_ID}*.nc')
            
        elif GCM_ in ['CanESM2', 'CESM1', 'GFDL-ESM2M']:
            hist_paths = glob.glob(
                f'/glade/collections/cdg/data/CLIVAR_LE/{model_center[GCM_]}/'\
                f'OImon/sic/*historical_rcp85_{var_ID}*.nc')
            fut_paths = ['foo']
        
        elif GCM_ == 'MPI-ESM1':
            hist_paths = glob.glob(
                f'/glade/collections/cdg/data/CLIVAR_LE/{model_center[GCM_]}/'\
                f'OImon/sic/*historical_{var_ID[:4]}*.nc')
            fut_paths = glob.glob(
                f'/glade/collections/cdg/data/CLIVAR_LE/{model_center[GCM_]}/'\
                f'OImon/sic/*rcp85_{var_ID[:4]}*.nc')
            
        else:
            base_hist = '/glade/collections/cmip/CMIP6/CMIP/'\
               +f'{model_center[GCM_]}/{GCM_}/historical/{var_ID}/SImon/'\
               +f'{var_name}'
            
            grid_ID = os.listdir(base_hist)[0]
            version = os.listdir(base_hist+f'/{grid_ID}/')[0]
            
            hist_paths = glob.glob(
                base_hist+f'/{grid_ID}/{version}/{var_name}/*.nc')
            
            base_ssp370 = '/glade/collections/cmip/CMIP6/ScenarioMIP/'\
               +f'{model_center[GCM_]}/{GCM_}/ssp370/{var_ID}/SImon/'\
               +f'{var_name}'
            
            grid_ID = os.listdir(base_ssp370)[0]
            version = os.listdir(base_ssp370+f'/{grid_ID}/')[0]
            
            fut_paths = glob.glob(
                base_ssp370+f'/{grid_ID}/{version}/{var_name}/*.nc')
            
        ############# do the regional and pan-Arctic computation ###############

        with dask.config.set(**{'array.slicing.split_large_chunks': True}):
            to_compute = regional_calc_dask(
                GCM_, var_ID, areacello_with_nan, region_mask, hist_paths, 
                fut_paths
            )
        
        computed = to_compute.chunk(chunks={'region':1,'time':120}).compute()

        #save to netCDF with attributes
        computed.attrs = {            
            'Description': 'Regional sea ice concentration/area/extent in % '\
                +'and pan-Arctic sea ice area/extent in million square km '\
                +f'for the climate model {GCM_} for variant ID {var_ID} for '\
                +'each month for the historical period until 2100 following '\
                +'RCP8.5 or ssp370 forcing. Regions as defined for NSIDC '\
                +'MASIE-NH Version 1, doi:10.7265/N5GT5K3K.',
            'Timestamp'  : str(datetime.datetime.utcnow().strftime(
                "%H:%M UTC %a %Y-%m-%d")),
        }

        computed.to_netcdf('/glade/work/cwpowell/ice-free/sea_ice_data/'\
           f'SIA_SIE_{GCM_}_{var_ID}_historical_2100.nc')
        